In [61]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
import statsmodels.api as sm

In [62]:
# import feature data
data = pd.read_csv('../data/processed_data/Wildfire_Weather_2020_2024.csv')

In [63]:
# view feature data
data.head()

,unique_id,size (km2),perimiter (km),startdate,enddate,duration,fire_line (km),fire_spread (km2/day),fire_speed (km/day),dominant_direction,...,d2m,t2m,msl,sp,lai_hv,lai_lv,tp,ssr,datetime,dist
0,2020_22784,4.93,11.11,2020-01-06 00:00:00,2020-01-13 00:00:00,8,1.1,0.62,0.88,southeast,...,278.64328,284.38715,102901.520,102447.460,2.988195,2.407728,0.000021,8017088.0,2020-01-09,6565.323606
1,2020_22784,4.93,11.11,2020-01-06 00:00:00,2020-01-13 00:00:00,8,1.1,0.62,0.88,southeast,...,287.59494,292.01648,102516.750,102070.190,2.986613,2.406662,0.000020,5444096.0,2020-01-10,6565.323606
2,2020_22784,4.93,11.11,2020-01-06 00:00:00,2020-01-13 00:00:00,8,1.1,0.62,0.88,southeast,...,287.74893,289.71127,102244.630,101797.350,2.983510,2.404584,0.003486,8655968.0,2020-01-12,6565.323606
3,2020_22784,4.93,11.11,2020-01-06 00:00:00,2020-01-13 00:00:00,8,1.1,0.62,0.88,southeast,...,278.82760,283.12076,102448.375,101995.414,2.992889,2.410894,0.000000,12236688.0,2020-01-06,6565.323606
4,2020_22784,4.93,11.11,2020-01-06 00:00:00,2020-01-13 00:00:00,8,1.1,0.62,0.88,southeast,...,292.18143,294.35640,101801.750,101360.100,2.985026,2.405601,0.025873,2535104.0,2020-01-11,6565.323606


In [64]:
# view feature data
data.describe()

,size (km2),perimiter (km),duration,fire_line (km),fire_spread (km2/day),fire_speed (km/day),index_right,latitude,longitude,u10,...,v10,d2m,t2m,msl,sp,lai_hv,lai_lv,tp,ssr,dist
count,32082.000000,32082.000000,32082.000000,32082.000000,32082.000000,32082.00000,3.208200e+04,32082.000000,32082.000000,32082.000000,...,32082.000000,32082.000000,32082.000000,32082.000000,32082.000000,32082.000000,32082.000000,32082.000000,3.208200e+04,32082.000000
mean,19.627479,29.133438,12.941774,2.496167,1.620429,2.20623,1.410558e+06,38.488632,-103.098714,0.128195,...,0.341190,280.752792,290.498979,101587.466259,95404.544994,2.717593,1.764379,0.001590,1.680758e+07,9411.910052
std,49.696907,27.715941,5.844891,3.886487,4.010768,1.70552,6.795278e+05,8.861038,18.942586,1.832064,...,2.349705,8.882896,7.656374,614.691170,7389.700655,1.142001,0.716741,0.004740,5.577529e+06,3635.845120
min,4.070000,9.260000,1.000000,0.350000,0.170000,0.58000,5.014000e+03,19.650000,-165.850000,-9.022443,...,-10.041690,247.640150,256.153400,99178.340000,70101.730000,0.000000,0.000000,0.000000,4.577280e+05,250.207361
25%,5.360000,15.740000,8.000000,0.930000,0.450000,1.30000,8.644580e+05,33.150000,-114.850000,-1.051346,...,-1.024666,274.147685,285.338530,101231.480000,91950.432500,2.030273,1.265003,0.000000,1.331969e+07,6686.224160
50%,7.930000,20.370000,12.000000,1.470000,0.730000,1.73000,1.391799e+06,36.650000,-96.600000,0.174160,...,0.299106,280.678035,290.937680,101582.675000,97986.463000,2.593112,1.686295,0.000029,1.739407e+07,9681.550126
75%,14.150000,31.480000,17.000000,2.620000,1.410000,2.49000,1.957546e+06,42.900000,-90.600000,1.194915,...,1.707381,287.887285,296.530053,101939.154250,100956.050000,3.549260,2.393430,0.000806,2.063463e+07,12217.278808
max,806.240000,298.170000,31.000000,140.600000,134.500000,26.19000,2.923831e+06,68.650000,-74.350000,11.458554,...,10.673229,299.197050,311.581670,104212.430000,103518.164000,5.127843,3.819295,0.142104,2.985043e+07,17855.839385


In [65]:
# view feature data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32082 entries, 0 to 32081
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   unique_id              32082 non-null  object 
 1   size (km2)             32082 non-null  float64
 2   perimiter (km)         32082 non-null  float64
 3   startdate              32082 non-null  object 
 4   enddate                32082 non-null  object 
 5   duration               32082 non-null  int64  
 6   fire_line (km)         32082 non-null  float64
 7   fire_spread (km2/day)  32082 non-null  float64
 8   fire_speed (km/day)    32082 non-null  float64
 9   dominant_direction     32082 non-null  object 
 10  geometry               32082 non-null  object 
 11  burn_start             32082 non-null  object 
 12  burn_end               32082 non-null  object 
 13  index_right            32082 non-null  int64  
 14  valid_time             32082 non-null  object 
 15  la

In [66]:
# extract year from startdate
data['year'] = pd.to_datetime(data['startdate'], errors='coerce').dt.year

In [67]:
# split data
train_data = data[data['year'].isin([2020, 2021, 2022])]
test_data = data[data['year'].isin([2023, 2024])]

In [68]:
# Predictors and Targets
predictors = ['duration', 'u10', 'v10', 'd2m', 't2m', 'msl', 'sp', 'lai_hv', 'lai_lv', 'tp', 'ssr']
target1 = 'size (km2)'
target2 = 'fire_spread (km2/day)'

In [69]:
# Split the data
X_train = train_data[predictors]
y1_train = train_data[target1]
y2_train = train_data[target2]

In [70]:
# Set seed and CV strategy
random_seed = 1
kf = KFold(n_splits=5, shuffle=True, random_state=random_seed)

In [71]:
# Model 1: Predict size (km2)
# intialize the model
model1 = LinearRegression()

In [72]:
# Cross-validation: size
neg_mse1 = cross_val_score(model1, X_train, y1_train, cv=kf, scoring='neg_mean_squared_error')
rmse1 = np.sqrt(-neg_mse1)

In [ ]:
# Statsmodels for coeffs/p-values and R²
X1_const = sm.add_constant(X_train)
ols1 = sm.OLS(y1_train, X1_const).fit()

print("\nCoefficients and p-values (Model 1):")
print(ols1.summary())

print("Full Model 1: Predicting size (km2)")
# Print only R-squared
print(f"\nR² for Model 1 (OLS fit): {ols1.rsquared:.4f}")
print(f"Correlation coefficient (r): {ols1.rsquared**0.5:.4f}")
# compute the mean RMSE for the CV
print(f"5-fold CV Mean MSE: {-neg_mse1.mean()}")
print(f"5-fold CV Mean RMSE: {rmse1.mean():.4f}")



Coefficients and p-values (Model 1):
                            OLS Regression Results                            
Dep. Variable:             size (km2)   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     414.8
Date:                Sun, 29 Jun 2025   Prob (F-statistic):               0.00
Time:                        12:57:27   Log-Likelihood:            -1.3751e+05
No. Observations:               25917   AIC:                         2.750e+05
Df Residuals:                   25905   BIC:                         2.751e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         

In [74]:
# Model 2: Predict fire_spread (km2/day)
model2 = LinearRegression()

In [75]:
neg_mse2 = cross_val_score(model2, X_train, y2_train, cv=kf, scoring='neg_mean_squared_error')
rmse2 = np.sqrt(-neg_mse2)

In [76]:
# Statsmodels summary for second model
X2_const = sm.add_constant(X_train)
ols2 = sm.OLS(y2_train, X2_const).fit()
print("\nCoefficients and p-values (Model 2):")
print(ols2.summary())
# Print only R-squared
print(f"\nR² for Model 2 (OLS fit): {ols2.rsquared:.4f}")
print(f"Correlation coefficient (r): {ols2.rsquared**0.5:.4f}")
print(f"5-fold CV Mean MSE: {-neg_mse2.mean()}")
print(f"5-fold CV Mean RMSE: {rmse2.mean():.4f}")


Coefficients and p-values (Model 2):
                              OLS Regression Results                             
Dep. Variable:     fire_spread (km2/day)   R-squared:                       0.125
Model:                               OLS   Adj. R-squared:                  0.125
Method:                    Least Squares   F-statistic:                     337.6
Date:                   Sun, 29 Jun 2025   Prob (F-statistic):               0.00
Time:                           12:57:27   Log-Likelihood:                -64796.
No. Observations:                  25917   AIC:                         1.296e+05
Df Residuals:                      25905   BIC:                         1.297e+05
Df Model:                             11                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [77]:
def iterative_feature_elimination(X, y, alpha=0.05):
    X_const = sm.add_constant(X)
    ols = sm.OLS(y, X_const).fit()

    while True:
        pvalues = ols.pvalues.drop('const')
        max_p = pvalues.max()
        if max_p <= alpha:
            break  # All predictors significant enough

        # Remove predictor with max p-value
        worst_feature = pvalues.idxmax()
        print(f"Removing '{worst_feature}' with p-value {max_p:.4f}")
        X = X.drop(columns=[worst_feature])

        X_const = sm.add_constant(X)
        ols = sm.OLS(y, X_const).fit()

    return X, ols

# For Model 1
print("\n--- Iterative elimination Model 1 ---")
X1_reduced, ols1_reduced = iterative_feature_elimination(X_train, y1_train)

# Recompute CV RMSE with reduced predictors
model1_reduced = LinearRegression()
kf = KFold(n_splits=5, shuffle=True, random_state=42)
neg_mse1_reduced = cross_val_score(model1_reduced, X1_reduced, y1_train, cv=kf, scoring='neg_mean_squared_error')
rmse1_reduced = np.sqrt(-neg_mse1_reduced)

print(f"\nReduced Model 1 predictors: {list(X1_reduced.columns)}")

print("\nCoefficients and p-values after elimination (Model 1):")
print(ols1_reduced.summary())
print(f"R² for Reduced Model 1: {ols1_reduced.rsquared:.4f}")
print(f"Correlation coefficient (r): {ols1_reduced.rsquared**0.5:.4f}")
print(f"5-fold CV Mean RMSE (reduced Model 1): {-neg_mse1_reduced.mean():.4f}")
print(f"5-fold CV Mean RMSE (reduced Model 1): {rmse1_reduced.mean():.4f}")

# For Model 2
print("\n--- Iterative elimination Model 2 ---")
X2_reduced, ols2_reduced = iterative_feature_elimination(X_train, y2_train)

# Recompute CV RMSE with reduced predictors
model2_reduced = LinearRegression()
neg_mse2_reduced = cross_val_score(model2_reduced, X2_reduced, y2_train, cv=kf, scoring='neg_mean_squared_error')
rmse2_reduced = np.sqrt(-neg_mse2_reduced)

print(f"\nReduced Model 2 predictors: {list(X2_reduced.columns)}")

print("\nCoefficients and p-values after elimination (Model 2):")
print(ols2_reduced.summary())
print(f"R² for Reduced Model 2: {ols2_reduced.rsquared:.4f}")
print(f"Correlation coefficient (r): {ols2_reduced.rsquared**0.5:.4f}")
print(f"5-fold CV Mean RMSE (reduced Model 2): {-neg_mse2_reduced.mean():.4f}")
print(f"5-fold CV Mean RMSE (reduced Model 2): {rmse2_reduced.mean():.4f}")



--- Iterative elimination Model 1 ---
Removing 'u10' with p-value 0.2715
Removing 'v10' with p-value 0.1553

Reduced Model 1 predictors: ['duration', 'd2m', 't2m', 'msl', 'sp', 'lai_hv', 'lai_lv', 'tp', 'ssr']

Coefficients and p-values after elimination (Model 1):
                            OLS Regression Results                            
Dep. Variable:             size (km2)   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     506.6
Date:                Sun, 29 Jun 2025   Prob (F-statistic):               0.00
Time:                        12:57:27   Log-Likelihood:            -1.3751e+05
No. Observations:               25917   AIC:                         2.750e+05
Df Residuals:                   25907   BIC:                         2.751e+05
Df Model:                           9                                         
Covariance Type:      